# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
%matplotlib inline
import pandas as pd
data = pd.read_csv('/datasets/data.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [2]:
data.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [3]:
#Посмотрим, как обстоят дела с целями.
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

In [4]:
# Образование
data['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

In [5]:
data['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

In [6]:
data['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

**Вывод**

В столбцах 'days_employed' и 'total_income' количество значений меньше, чем в остальных, а также в этих столбцах по 6 знаков после запятой. В столбце 'purpose' много похожих значений, например: "на проведение свадьбы", "сыграть свадьбу". Отрицательные значения в столбце 'children' будут скорее всего мешать. Заменим "-1" на "1", поскольку, скорее всего, здесь подразумевается 1 ребенок, и таких значений всего 47. В столбцах 'education' и 'family_status' необходимо разобраться с прописными буквами.

## Шаг 2. Предобработка данных

### Обработка пропусков

In [7]:
# Сначала посмотрим есть ли пропуски в данных.
data.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [8]:
# Обработаем пропуски. Для начала посмотрим как обстоят дела в столбце 'total_income' со средним и медианой 
# по категориям столбца 'income_type'
income_type_gr = data.groupby('income_type').agg({'total_income' : ['min', 'median', 'mean']})
print(income_type_gr)


                  total_income                              
                           min         median           mean
income_type                                                 
безработный       59956.991984  131339.751676  131339.751676
в декрете         53829.130729   53829.130729   53829.130729
госслужащий       29200.077193  150447.935283  170898.309923
компаньон         28702.812889  172357.950966  202417.461462
пенсионер         20667.263793  118514.486412  137127.465690
предприниматель  499163.144947  499163.144947  499163.144947
сотрудник         21367.648356  142594.396847  161380.260488
студент           98201.625314   98201.625314   98201.625314


In [9]:
data['total_income'] = data.groupby('income_type')['total_income'].apply(lambda x: x.fillna(x.median()))
data.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [10]:
# Обработаем аналогично пропуски в столбце 'days_employed'

days_employed_group = data.groupby('dob_years').agg({'days_employed' : ['min', 'median', 'mean']})
print(days_employed_group.head())

          days_employed                           
                    min       median          mean
dob_years                                         
0         -10689.250498 -1146.689586  65937.471974
19         -1020.183130  -724.492610   -633.678086
20         -1547.804060  -674.838979   -684.944308
21         -1948.929810  -618.733817   -709.440930
22         -2106.520236  -698.101581   1239.982481


In [11]:
data['days_employed'] = data.groupby('dob_years')['days_employed'].apply(lambda x: x.fillna(x.mean()))
data.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


**Вывод**

В столбцах 'days_employed' и 'total_income' есть несуществующие значения. Таких значений по 2174 в каждом столбце. 

### Замена типа данных

In [12]:
# Уберем прописные буквы из столбцов "Семейное положение".
data['family_status'] = data['family_status'].str.lower()
data['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

In [13]:
# Избавимся от отрицательных значений в столбце "Количество детей". 

data.loc[data['children'] == -1, 'children'] = 1
data['children'].value_counts()

0     14149
1      4865
2      2055
3       330
20       76
4        41
5         9
Name: children, dtype: int64

In [14]:
# Уберем прописные буквы в столбце "Образование".
data['education'] = data['education'].str.lower()
data['education'].value_counts()

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

In [15]:
# В столбце "Ежемесячный доход" избавимся от дробных чисел. 
data['total_income'] = data['total_income'].astype('int')
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,-5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


In [16]:
data['days_employed'] = pd.to_numeric(data['days_employed'])
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,-5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


**Вывод**

Все типы данных соответствуют содержимому. Но столбцы "Общий трудовой стаж" и "Ежемесячный доход" имеют по 6 знаков после запятой. В обоих столбцах  можно смело менять числа на целые, поскольку стаж считается в днях, а в доходе копейки не изменят общей картины. В столбце "Ежемесячный доход" заменим тип данных методом .astype(). Изменим регистр в столбцах 'education' и 'family_status' методом str.lower(). А также избавимся от отрицательных значений в столбце 'children' с помощью логической индексации. 

### Обработка дубликатов

In [17]:
#Посчитаем дубликаты.
data.duplicated().sum()

71

In [18]:
data['total_income'].value_counts()

142594    1105
172357     509
118514     414
150447     147
126262       3
          ... 
101387       1
138249       1
280240       1
390148       1
264193       1
Name: total_income, Length: 18608, dtype: int64

In [19]:
data['days_employed'].value_counts()

 196.083153       69
 2446.420996      66
-967.511501       65
 3393.631090      65
-1454.909683      64
                  ..
 381198.918686     1
 367794.325156     1
-289.563679        1
-257.733034        1
-582.538413        1
Name: days_employed, Length: 19407, dtype: int64

**Вывод**

Всего найден 71 дубликат. Поскольку у нас нет идентификационных данных, сказать точно, что это дубликаты, мы не можем. Данные разных людей могут просто совпасть.

### Лемматизация

In [20]:
# Приведем категории столбца "Цель кредита" в более удобный для дальнейшей обработки вид.
from pymystem3 import Mystem
m = Mystem()
def lemmas_purpose(row):
    lemmas = m.lemmatize(row)
    return lemmas
data['purpose_lem'] = data['purpose'].apply(lemmas_purpose)
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lem
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,"[покупка, , жилье, \n]"
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]"
2,0,-5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,"[покупка, , жилье, \n]"
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,"[дополнительный, , образование, \n]"
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]"


In [21]:
data['purpose_lem'].value_counts()


[автомобиль, \n]                                          973
[свадьба, \n]                                             797
[на,  , проведение,  , свадьба, \n]                       777
[сыграть,  , свадьба, \n]                                 774
[операция,  , с,  , недвижимость, \n]                     676
[покупка,  , коммерческий,  , недвижимость, \n]           664
[операция,  , с,  , жилье, \n]                            653
[покупка,  , жилье,  , для,  , сдача, \n]                 653
[операция,  , с,  , коммерческий,  , недвижимость, \n]    651
[покупка,  , жилье, \n]                                   647
[жилье, \n]                                               647
[покупка,  , жилье,  , для,  , семья, \n]                 641
[строительство,  , собственный,  , недвижимость, \n]      635
[недвижимость, \n]                                        634
[операция,  , со,  , свой,  , недвижимость, \n]           630
[строительство,  , жилой,  , недвижимость, \n]            626
[покупка

**Вывод**

Мы упростили цели кредита. Их можно свести до категорий: "автомобиль", "ипотека", "ремонт жилья", "образование", "свадьба", "коммерческая недвижимость", "строительство".

### Категоризация данных

In [22]:
def categ_purpose(row):
    if 'строительство' in row:
        return 'строительство'
    if 'ремонт' in row:
        return 'ремонт жилья'
    if 'коммерческий' in row:
        return 'коммерческая недвижимость'
    if 'жилье' in row:
        return 'ипотека'
    if 'жилой' in row:
        return 'ипотека'
    if 'недвижимость' in row:
        return 'ипотека'
    if 'образование' in row:
        return 'образование'
    if 'автомобиль' in row:
        return 'покупка автомобиля'
    if 'свадьба' in row:
        return 'свадьба'
data['purpose_cat'] = data['purpose_lem'].apply(categ_purpose)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 14 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
purpose_lem         21525 non-null object
purpose_cat         21525 non-null object
dtypes: float64(1), int64(6), object(7)
memory usage: 2.3+ MB


In [23]:
# Проверим результат
data['purpose_cat'].value_counts()

ипотека                      7032
покупка автомобиля           4315
образование                  4022
свадьба                      2348
строительство                1881
коммерческая недвижимость    1315
ремонт жилья                  612
Name: purpose_cat, dtype: int64

In [24]:
education_voc = data.groupby('education_id')['education'].unique()
print(education_voc)

education_id
0                 [высшее]
1                [среднее]
2    [неоконченное высшее]
3              [начальное]
4         [ученая степень]
Name: education, dtype: object


In [25]:
family_status_voc = data.groupby('family_status_id')['family_status'].unique()
print(family_status_voc)

family_status_id
0          [женат / замужем]
1         [гражданский брак]
2           [вдовец / вдова]
3                [в разводе]
4    [не женат / не замужем]
Name: family_status, dtype: object


**Вывод**

Категоризация данных прошла успешно: теперь у нас есть столбец, в котором цели кредита сведены к семи категориям. Метод .info() позволяет увидеть, что все столбцы имеют правильный тип данных и заполнены полностью. Можем переходить непосредственно к анализу. Сразу можем отметить, что больше всего кредитов берут на покупку жилья.

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [26]:
children = data['children'].value_counts()
children_vs_credit = data.groupby('children')['debt'].sum()
convers_child = ((children_vs_credit / children * 10000).astype('int')) / 100
print(convers_child.sort_values(ascending = True))


5      0.00
0      7.51
3      8.18
1      9.14
2      9.44
4      9.75
20    10.52
dtype: float64


**Вывод**

Нельзя сказать о прямой зависимости количества невозвратов от количества детей. В семьях с 5 детьми нет должников, похоже на артефакт. В семьях без детей процент задолжности меньше, но с учетом того, что таких заемщиков большинство, стоит посмотреть не в процентном соотношении, а в количественном, например, в сводной таблице с учетом семейного положения или дохода.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [27]:
family = data['family_status'].value_counts()
family_vs_kredit = data.groupby('family_status')['debt'].sum()
convers_family = (family_vs_kredit / family * 10000).astype('int') / 100
print(convers_family.sort_values(ascending = True))

вдовец / вдова           6.56
в разводе                7.11
женат / замужем          7.52
гражданский брак         9.28
не женат / не замужем    9.74
dtype: float64


In [28]:
family_vs_kredit_agg = data.groupby('family_status').agg({'debt' : ['sum', 'count', 'mean']})
print(family_vs_kredit_agg)

                      debt                 
                       sum  count      mean
family_status                              
в разводе               85   1195  0.071130
вдовец / вдова          63    960  0.065625
гражданский брак       388   4177  0.092890
женат / замужем        931  12380  0.075202
не женат / не замужем  274   2813  0.097405


**Вывод**

Люди, имеющие или имевшие официальный брак, ответственнее и процент задолжностей у них меньше. В то время как люди, не оформившие отношения официально или не имеющие партнера, чаще имеют задолжность.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [29]:
# Узнаем максимальный доход
data['total_income'].max()

2265604

In [30]:
# средний
data['total_income'].mean()

165224.81602787456

In [31]:
# минимальный
data['total_income'].min()

20667

In [32]:
# и медиану
data['total_income'].median()

142594.0

In [33]:
# Распределим по равным группам на 9 категорий.
pd.qcut(data['total_income'], q=9)

0        (204354.444, 261590.333]
1        (102616.556, 119245.667]
2            (140523.0, 151405.0]
3         (261590.333, 2265604.0]
4            (151405.0, 172357.0]
                   ...           
21520    (204354.444, 261590.333]
21521        (151405.0, 172357.0]
21522     (81047.111, 102616.556]
21523    (204354.444, 261590.333]
21524     (81047.111, 102616.556]
Name: total_income, Length: 21525, dtype: category
Categories (9, interval[float64]): [(20666.999, 81047.111] < (81047.111, 102616.556] < (102616.556, 119245.667] < (119245.667, 140523.0] ... (151405.0, 172357.0] < (172357.0, 204354.444] < (204354.444, 261590.333] < (261590.333, 2265604.0]]

In [34]:
# Разобьем доходы на группы
def total_income_group(total):
    if total > 260000:
        return 'доход более 260 тыс.'
    if total > 205000:
        return 'доход от 205 до 260 тыс.'
    if total > 173000:
        return 'доход от 173 до 205 тыс.'
    if total > 150000:
        return 'доход от 150 до 173 тыс.'
    if total > 140000:
        return 'доход от 140 до 150 тыс.'
    if total > 120000:
        return 'доход от 120 до 140 тыс.'
    if total > 105000:
        return 'доход от 105 до 120 тыс.'
    if total > 81000:
        return 'доход от 80 до 105 тыс.'
    return 'доход менее 80 тыс.'
data['total_income_gr'] = data['total_income'].apply(total_income_group)
data.head(10) # и проверим корректность функции

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lem,purpose_cat,total_income_gr
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,"[покупка, , жилье, \n]",ипотека,доход от 205 до 260 тыс.
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]",покупка автомобиля,доход от 105 до 120 тыс.
2,0,-5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,"[покупка, , жилье, \n]",ипотека,доход от 140 до 150 тыс.
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,"[дополнительный, , образование, \n]",образование,доход более 260 тыс.
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]",свадьба,доход от 150 до 173 тыс.
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,"[покупка, , жилье, \n]",ипотека,доход от 205 до 260 тыс.
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,"[операция, , с, , жилье, \n]",ипотека,доход от 205 до 260 тыс.
7,0,-152.779569,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,"[образование, \n]",образование,доход от 120 до 140 тыс.
8,2,-6929.865299,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,"[на, , проведение, , свадьба, \n]",свадьба,доход от 80 до 105 тыс.
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,"[покупка, , жилье, , для, , семья, \n]",ипотека,доход от 140 до 150 тыс.


In [35]:
data['total_income_gr'].value_counts()

доход от 150 до 173 тыс.    2877
доход от 80 до 105 тыс.     2694
доход более 260 тыс.        2459
доход менее 80 тыс.         2383
доход от 205 до 260 тыс.    2276
доход от 120 до 140 тыс.    2260
доход от 173 до 205 тыс.    2228
доход от 105 до 120 тыс.    2181
доход от 140 до 150 тыс.    2167
Name: total_income_gr, dtype: int64

In [36]:
income = data['total_income_gr'].value_counts()
income_vs_credit = data.groupby('total_income_gr')['debt'].sum()
convers_total_income = (income_vs_credit / income * 10000).astype('int') / 100
print(convers_total_income.sort_values(ascending = True))

доход более 260 тыс.        6.83
доход от 205 до 260 тыс.    7.11
доход менее 80 тыс.         7.59
доход от 80 до 105 тыс.     8.05
доход от 150 до 173 тыс.    8.34
доход от 105 до 120 тыс.    8.61
доход от 120 до 140 тыс.    8.62
доход от 173 до 205 тыс.    8.66
доход от 140 до 150 тыс.    9.09
dtype: float64


**Вывод**

Уровень дохода колеблется от 20667 до 2 265 604. Чтобы понять насколько возврат кредита в срок зависит от уровня дохода, нам необходимо сгруппировать данные в зависимости от уровня дохода, соответственно разбить на несколько категорий. Сгруппируем данные по новому столбцу и посчитаем количество и процент задолжностей в каждой группе. Больше всего задолжностей при уровне дохода от 140 до 150 тысяч. Самый маленький процент задолжностей при уровне дохода более 260 тысяч. 

- Как разные цели кредита влияют на его возврат в срок?

In [37]:
purpose_category = data['purpose_cat'].value_counts()
purpose_vs_credit = data.groupby('purpose_cat')['debt'].sum()
convers_purpose = (purpose_vs_credit / purpose_category * 10000).astype('int') / 100
print(convers_purpose.sort_values(ascending = True))

ремонт жилья                 5.71
ипотека                      7.16
коммерческая недвижимость    7.52
строительство                7.65
свадьба                      7.92
образование                  9.19
покупка автомобиля           9.33
dtype: float64


**Вывод**

Кредиты, взятые на ремонт жилья возвращаются лучше, а на получение образования и покупку автомобиля хуже.

## Шаг 4. Общий вывод

Сложно судить по одной категории: определенные закономерности прослеживаются, но многое неясно. Составим сводные таблицы, посчитаем количество задолжностей и кредитов в целом.

In [38]:
# Количество задолжностей в зависимости от семейного положения и наличия детей
data_pivot_fam_child_s = data.pivot_table(index = 'family_status', columns = 'children', values = 'debt', aggfunc = 'sum')
print(data_pivot_fam_child_s)

children                  0      1      2     3    4    5    20
family_status                                                  
в разводе               55.0   21.0    7.0   1.0  0.0  NaN  1.0
вдовец / вдова          53.0    7.0    3.0   0.0  0.0  NaN  0.0
гражданский брак       229.0  118.0   30.0   8.0  0.0  0.0  3.0
женат / замужем        516.0  247.0  145.0  17.0  3.0  0.0  3.0
не женат / не замужем  210.0   52.0    9.0   1.0  1.0  NaN  1.0


In [39]:
# Количество кредитов в зависимости от семейного положения и наличия детей
data_pivot_fam_child_c = data.pivot_table(index = 'family_status', columns = 'children', values = 'debt', aggfunc = 'count')
print(data_pivot_fam_child_c)

children                   0       1       2      3     4    5     20
family_status                                                        
в разводе               784.0   316.0    81.0   11.0   1.0  NaN   2.0
вдовец / вдова          848.0    81.0    20.0    6.0   1.0  NaN   4.0
гражданский брак       2752.0  1003.0   344.0   56.0   8.0  2.0  12.0
женат / замужем        7500.0  3011.0  1535.0  249.0  29.0  7.0  49.0
не женат / не замужем  2265.0   454.0    75.0    8.0   2.0  NaN   9.0


In [40]:
data_pivot_pur_fam = data.pivot_table(index = 'purpose_cat', columns = 'family_status_id', values = 'debt', aggfunc = 'sum')
print(data_pivot_pur_fam)

family_status_id               0      1     2     3     4
purpose_cat                                              
ипотека                    316.0   58.0  18.0  27.0  85.0
коммерческая недвижимость   60.0   11.0   5.0   6.0  17.0
образование                216.0   60.0  15.0  17.0  62.0
покупка автомобиля         229.0   51.0  20.0  21.0  82.0
ремонт жилья                19.0    3.0   2.0   5.0   6.0
свадьба                      NaN  186.0   NaN   NaN   NaN
строительство               91.0   19.0   3.0   9.0  22.0


In [41]:
data_pivot_pur_fam = data.pivot_table(index = 'purpose_cat', columns = 'family_status_id', values = 'debt', aggfunc = 'count')
print(data_pivot_pur_fam)

family_status_id                0       1      2      3       4
purpose_cat                                                    
ипотека                    4592.0   631.0  339.0  433.0  1037.0
коммерческая недвижимость   838.0   123.0   77.0   92.0   185.0
образование                2603.0   404.0  199.0  238.0   578.0
покупка автомобиля         2744.0   434.0  218.0  281.0   638.0
ремонт жилья                393.0    55.0   32.0   39.0    93.0
свадьба                       NaN  2348.0    NaN    NaN     NaN
строительство              1210.0   182.0   95.0  112.0   282.0


Сводные таблицы в целом подтверждают выводы, сделанные ранее.
В семьях без детей процент задолжности меньше, но кредиты берут гораздо чаще, соответственно по количеству задолжностей гораздо больше. Семейные люди в целом пользуются кредитами чаще одиноких.
Люди, имеющие или имевшие официальный брак, ответственнее и процент задолжностей у них меньше. В то время как люди, не оформившие отношения официально или не имеющие партнера, чаще имеют задолжность. И только живущие в гражданском браке имеют задолжности по кредиту, взятому на свадьбу. Как показывает последняя таблица, только они такие кредиты и берут.
